# Data processing for XLSTM-CEEMDAN model

## Python requirements

```
!pip install plotly
!pip install cufflinks
!pip install chart_studio
!pip install ipywidgets
!pip install yfinance
!pip install EMD-signal
!pip install sklearn
!pip install keras
!pip install tensorflow
```

## Model

The **XLSTM-CEEMDAN** works with daily data from equities: open, high, low, close, volume. Hereinafter we are calling these "features".

### Data processing steps:

1. Data gathering

We use yfinance package to get the daily ticker history. We obtain a dataframe with one time series for each of the features, for each of the tickers we are working with.

Here we are considering the top 10 most liquid tickers in Brazilian B3 exchange in period of **2019-01-01 until 2020-05-01**:

**PETR4, VALE3, BOVA11, ITUB4, BBDC4, B3SA3, BBAS3, ABEV3, MGLU3, VVAR3**

2. Decomposition

We decompose the data with the complete ensemble empirical mode decomposition with adative noise (CEEMDAN) algorithm, using the [PyEMD public package from Dawid Laszuk](github.com/laszukdawid/PyEMD).

As a result, we obtain a set of instrinsic mode functions (IMFs) time series for each of the features, for each of the tickers. The number of IMFs resulting from a decomposition may vary, and depends upon CEEMDAN hyperparameters such as scale of added noise, specific series characteristics, and mainly **series length**.

3. Data transformation

For each of the IMFs series obtained, we tranform the data according to the following equation in order to work with data values between 0 and 1.

$$x' = \frac{x-x_{min}}{x_{max}-x_{min}}$$

Where x is any element in the series to be transformed, x' is the equivalent transformed element. The equation also takes into account the maximum and minimum values of the series.

4. Windowing

Besides the mathematical transformation, each series undergo a vectorial transformation in order to split the data into windows.

The process is examplified:

```
original single dimensional series = [1,2,3,4,5,6,7,8]
Splitting into windows of length 4.
Windowed data = [[1,2,3,4],[2,3,4,5],[3,4,5,6],[4,5,6,7],[5,6,7,8]]
```
This way the resulting series will have $n-w+1$ elements, where $n$ is the length of the orginal series and $w$ is the window length.

### Processed data availability

After the processing method, the final data is available in a dataframe format.

In [1]:
from datetime import timedelta, datetime
import pandas as pd
import numpy as np

import cufflinks as cf
import chart_studio.plotly as plotly
import plotly.offline
cf.go_offline()
cf.set_config_file(offline=True, world_readable=False)

from sklearn.preprocessing import MinMaxScaler
from keras.preprocessing.sequence import TimeseriesGenerator
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
import yfinance as yf
from PyEMD import CEEMDAN

# iplot layout
space =  {
            'legend' : {'bgcolor':'#1A1A1C','font':{'color':'#D9D9D9',"size":12}},
            'paper_bgcolor' : '#1A1A1C',
            'plot_bgcolor' : '#1A1A1C',
            "title" : {"font":{"color":"#D9D9D9"},"x":0.5},
            'yaxis' : {
                'tickfont' : {'color':'#C2C2C2', "size":12},
                'gridcolor' : '#434343',
                'titlefont' : {'color':'#D9D9D9'},
                'zerolinecolor' : '#666570',
                'showgrid' : True
            },
            'xaxis' : {
                'tickfont' : {'color':'#C2C2C2', "size":12},
                'gridcolor' : '#434343',
                'titlefont' : {'color':'#D9D9D9'},
                'zerolinecolor' : '#666570',
                'showgrid' : True
            },
            'titlefont' : {'color':'#D9D9D9'}
        }

Using TensorFlow backend.


In [2]:
tickers = ["PETR4", "VALE3", "BOVA11", "ITUB4", "BBDC4", "B3SA3", "BBAS3", "ABEV3", "MGLU3", "VVAR3"]

tickers = [f"{ticker}.SA" for ticker in tickers]

start_datetime = datetime(year=2018, month=5, day=1)
end_datetime = datetime(year=2020, month=5, day=1)

history_data = {ticker.split('.')[0]:yf.download(ticker, start=start_datetime, end=end_datetime).drop(['Close'], axis=1).dropna() for ticker in tickers}

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [3]:
ceemdan = CEEMDAN()
decomposed_data = {}
decomposed_ticker_features_series = {}
scalers = {}
for ticker in history_data:
    # iterating every ticker
    print(f'[{ticker}] Decomposing...')
    ticker_dataframe = history_data[ticker]
    decomposed_ticker_features_series[ticker] = {}
    scalers[ticker] = {}
    for column in ticker_dataframe.columns:
        # iterating evey feature
        scaler = MinMaxScaler()
        decomposed_ticker_features_series[ticker][column] = {}
        series = ticker_dataframe[column].values.reshape(-1,1)
        scaler.fit(series)
        scalers[ticker][column] = scaler
        ticker_feature_time_series = np.frombuffer(scaler.transform(series))
        ticker_feature_time_series_imfs = ceemdan(ticker_feature_time_series, max_imf=10)
        for i, imf_series in enumerate(ticker_feature_time_series_imfs):
            # iterating every IMF 
            if i < len(ticker_feature_time_series_imfs):
                decomposed_ticker_features_series[ticker][column][f'IMF{i+1}'] = imf_series
            else:
                decomposed_ticker_features_series[ticker][column][f'Rsd'] = imf_series

[PETR4] Decomposing...
[VALE3] Decomposing...
[BOVA11] Decomposing...
[ITUB4] Decomposing...
[BBDC4] Decomposing...
[B3SA3] Decomposing...
[BBAS3] Decomposing...
[ABEV3] Decomposing...
[MGLU3] Decomposing...
[VVAR3] Decomposing...


In [4]:
# Data organisation
features_in_order = ['Open', 'High', 'Low', 'Adj Close', 'Volume']

windows_sizes_for_imf_level = {
    'IMF1': 2,
    'IMF2': 2,
    'IMF3': 3,
    'IMF4': 3,
    'IMF5': 4,
    'IMF6': 4,
    'IMF7': 5,
    'IMF8': 5,
    'Rsd': 6,
    'DEFAULT': 4
}

# Coupling together the IMFs of the same level for different features to create exogenous input
# The number of imfs for each feature decomposition may differ, thus some of the last imfs may not match in number of features
series = {}
for ticker in decomposed_ticker_features_series:
    series[ticker] = {}
    for feature in decomposed_ticker_features_series[ticker]:
        imfs = pd.DataFrame.from_dict(decomposed_ticker_features_series[ticker][feature])
        for imf in imfs:
            if imf not in series[ticker]:
                series[ticker][imf] = []
            _series = imfs[imf].values
            _series = _series.reshape((len(_series),1))
            series[ticker][imf] += [_series] # reshaping to get into column format

dataset = {}
# # horizontal stack
for ticker in series:
    dataset[ticker] = {}
    for imf_level in series[ticker]:
        dataset[ticker][imf_level] = np.hstack(tuple(series[ticker][imf_level]))

# # data set split rates
train = 0.7
validation = 0.2
test = 0.1

train_dataset = {}
validation_dataset = {}
test_dataset = {}

train_generators = {}
validation_generators = {}
test_generators = {}

for ticker in dataset:

    train_dataset[ticker] = {}
    validation_dataset[ticker] = {}
    test_dataset[ticker] = {}

    train_generators[ticker] = {}
    validation_generators[ticker] = {}
    test_generators[ticker] = {}

    for imf_level in dataset[ticker]:
        # splitting data sets according to rates
        train_dataset[ticker][imf_level] = dataset[ticker][imf_level][:round(train*dataset[ticker][imf_level].shape[0]),:]
        validation_dataset[ticker][imf_level] = dataset[ticker][imf_level][round(train*dataset[ticker][imf_level].shape[0]):round((train+validation)*dataset[ticker][imf_level].shape[0]),:]
        test_dataset[ticker][imf_level] = dataset[ticker][imf_level][round((train+validation)*dataset[ticker][imf_level].shape[0]):,:]

        if imf_level in windows_sizes_for_imf_level:
            window_size = windows_sizes_for_imf_level[imf_level]
        else: 
            window_size = windows_sizes_for_imf_level['DEFAULT']
        # windowing
        train_generators[ticker][imf_level] = TimeseriesGenerator(train_dataset[ticker][imf_level], train_dataset[ticker][imf_level], length=window_size, batch_size=1)
        validation_generators[ticker][imf_level] = TimeseriesGenerator(validation_dataset[ticker][imf_level], validation_dataset[ticker][imf_level], length=window_size, batch_size=1)
        test_generators[ticker][imf_level] = TimeseriesGenerator(test_dataset[ticker][imf_level], test_dataset[ticker][imf_level], length=window_size, batch_size=1)


In [5]:
cur_tmp_gen = train_generators['PETR4']['IMF1']
for i in range(len(cur_tmp_gen)):
	x, y = cur_tmp_gen[i]
	# print('%s => %s' % (x, y))
x.shape

(1, 2, 5)

In [6]:
# Model Training

models = {}

for ticker in train_generators:
    models[ticker] = {}
    for imf_level in train_generators[ticker]:
        print(f'Training model [{ticker}][{imf_level}]')
        # Prediction model
        model = Sequential()
        current_dataset = train_dataset[ticker][imf_level]
        n_features = current_dataset.shape[1]
        cur_tmp_gen = train_generators[ticker][imf_level]

        if imf_level in windows_sizes_for_imf_level:
            window_size = windows_sizes_for_imf_level[imf_level]
        else: 
            window_size = windows_sizes_for_imf_level['DEFAULT']

        model.add(LSTM(100, activation='relu', input_shape=(window_size, n_features)))
        model.add(Dense(n_features))
        model.compile(optimizer='adam', loss='mse')

        # fit model
        model.fit_generator(cur_tmp_gen, steps_per_epoch=1, epochs=1000, verbose=0)

        models[ticker][imf_level] = model

Training model [PETR4][IMF1]
Training model [PETR4][IMF2]
Training model [PETR4][IMF3]
Training model [PETR4][IMF4]
Training model [PETR4][IMF5]
Training model [PETR4][IMF6]
Training model [PETR4][IMF7]
Training model [PETR4][IMF8]
Training model [VALE3][IMF1]
Training model [VALE3][IMF2]
Training model [VALE3][IMF3]
Training model [VALE3][IMF4]
Training model [VALE3][IMF5]
Training model [VALE3][IMF6]
Training model [VALE3][IMF7]
Training model [BOVA11][IMF1]
Training model [BOVA11][IMF2]
Training model [BOVA11][IMF3]
Training model [BOVA11][IMF4]
Training model [BOVA11][IMF5]
Training model [BOVA11][IMF6]
Training model [BOVA11][IMF7]
Training model [BOVA11][IMF8]
Training model [ITUB4][IMF1]
Training model [ITUB4][IMF2]
Training model [ITUB4][IMF3]
Training model [ITUB4][IMF4]
Training model [ITUB4][IMF5]
Training model [ITUB4][IMF6]
Training model [ITUB4][IMF7]
Training model [ITUB4][IMF8]
Training model [BBDC4][IMF1]
Training model [BBDC4][IMF2]
Training model [BBDC4][IMF3]
Traini

In [7]:
results = {}

for ticker in models:
    results[ticker] = {}

    # initializing results dicitionary
    for feature in features_in_order:
        results[ticker][feature] = {}
        for imf_level in models[ticker]:
            results[ticker][feature][imf_level] = {
                'real_train': [],
                'predicted_train': [],
                'real_validation': [],
                'predicted_validation': [],
                'real_test': [],
                'predicted_test': [],
                'x_axis_train': [],
                'x_axis_validation': [],
                'x_axis_test': []
            }

    for imf_level in models[ticker]:
        model = models[ticker][imf_level]
        
        print(f'Predicting: [{ticker}][{imf_level}]')

        cur_train_gen = train_generators[ticker][imf_level]
        cur_validation_gen = validation_generators[ticker][imf_level]
        cur_test_gen = test_generators[ticker][imf_level]

        # predicting train
        day_counter = 0
        for i in range(len(cur_train_gen)):
            x, y = cur_train_gen[i]
            yhat = model.predict(x, verbose=0)

            for j in range(yhat.shape[1]):
                results[ticker][features_in_order[j]][imf_level]['real_train'] += [y[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['predicted_train'] += [yhat[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['x_axis_train'] += [day_counter]
            day_counter += 1

        # predicting validation
        for i in range(len(cur_validation_gen)):
            x, y = cur_validation_gen[i]
            yhat = model.predict(x, verbose=0)

            for j in range(yhat.shape[1]):
                results[ticker][features_in_order[j]][imf_level]['real_validation'] += [y[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['predicted_validation'] += [yhat[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['x_axis_validation'] += [day_counter]
            day_counter += 1

        # predicting test
        for i in range(len(cur_test_gen)):
            x, y = cur_test_gen[i]
            yhat = model.predict(x, verbose=0)

            for j in range(yhat.shape[1]):
                results[ticker][features_in_order[j]][imf_level]['real_test'] += [y[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['predicted_test'] += [yhat[:,j][0]]
                results[ticker][features_in_order[j]][imf_level]['x_axis_test'] += [day_counter]
            day_counter += 1


Predicting: [PETR4][IMF1]
Predicting: [PETR4][IMF2]
Predicting: [PETR4][IMF3]
Predicting: [PETR4][IMF4]
Predicting: [PETR4][IMF5]
Predicting: [PETR4][IMF6]
Predicting: [PETR4][IMF7]
Predicting: [PETR4][IMF8]
Predicting: [VALE3][IMF1]
Predicting: [VALE3][IMF2]
Predicting: [VALE3][IMF3]
Predicting: [VALE3][IMF4]
Predicting: [VALE3][IMF5]
Predicting: [VALE3][IMF6]
Predicting: [VALE3][IMF7]
Predicting: [BOVA11][IMF1]
Predicting: [BOVA11][IMF2]
Predicting: [BOVA11][IMF3]
Predicting: [BOVA11][IMF4]
Predicting: [BOVA11][IMF5]
Predicting: [BOVA11][IMF6]
Predicting: [BOVA11][IMF7]
Predicting: [BOVA11][IMF8]
Predicting: [ITUB4][IMF1]
Predicting: [ITUB4][IMF2]
Predicting: [ITUB4][IMF3]
Predicting: [ITUB4][IMF4]
Predicting: [ITUB4][IMF5]
Predicting: [ITUB4][IMF6]
Predicting: [ITUB4][IMF7]
Predicting: [ITUB4][IMF8]
Predicting: [BBDC4][IMF1]
Predicting: [BBDC4][IMF2]
Predicting: [BBDC4][IMF3]
Predicting: [BBDC4][IMF4]
Predicting: [BBDC4][IMF5]
Predicting: [BBDC4][IMF6]
Predicting: [BBDC4][IMF7]
Pred

In [8]:
# organizing imf prediction results, concatenating train, validation and test
concatenated_results = {}

for ticker in results:
    concatenated_results[ticker] = {}
    for feature in results[ticker]:
        concatenated_results[ticker][feature] = {}
        for imf_level in results[ticker][feature]:
            
            df_result = pd.DataFrame.from_dict(results[ticker][feature][imf_level], orient='index').T
            df_train = df_result[['real_train','predicted_train','x_axis_train']].set_index('x_axis_train').dropna(axis=0)
            df_train.index.name = 'x'
            df_validation = df_result[['real_validation','predicted_validation','x_axis_validation']].set_index('x_axis_validation').dropna(axis=0)
            df_validation.index.name = 'x'
            df_test = df_result[['real_test','predicted_test','x_axis_test']].set_index('x_axis_test').dropna(axis=0)
            df_test.index.name = 'x'

            df_concatenated = pd.concat([df_train,df_validation,df_test], axis=1)

            concatenated_results[ticker][feature][imf_level] = df_concatenated

In [9]:
# plotting partial result
plot_ticker = 'BBDC4'
plot_feature = 'High'
plot_imf = 'IMF3'

concatenated_results[plot_ticker][plot_feature][plot_imf].iplot(title=f'{plot_ticker} {plot_feature} {plot_imf}', asFigure=True, layout=space)

In [10]:
# recomposing prediction by arithmetically adding the IMF curves

final_prediction_results = {}

for ticker in concatenated_results:
    final_prediction_results[ticker] = {}
    for feature in concatenated_results[ticker]:
        addition_train = None
        addition_validation = None
        addition_test = None
        for imf_level in concatenated_results[ticker][feature]:
            # adding train
            if addition_train is None:
                addition_train = concatenated_results[ticker][feature][imf_level]['predicted_train'].values
            else:
                np_array_to_be_added = concatenated_results[ticker][feature][imf_level]['predicted_train'].values
                cur_length = addition_train.shape[0]
                next_np_array_length = np_array_to_be_added.shape[0]
                if cur_length < next_np_array_length:
                    np_array_to_be_added = np_array_to_be_added[next_np_array_length-cur_length:]
                else: 
                    addition_train = addition_train[cur_length-next_np_array_length:]
                addition_train = np.add(addition_train,np_array_to_be_added)

            # adding validation
            if addition_validation is None:
                addition_validation = concatenated_results[ticker][feature][imf_level]['predicted_validation'].values
            else:
                np_array_to_be_added = concatenated_results[ticker][feature][imf_level]['predicted_validation'].values
                cur_length = addition_validation.shape[0]
                next_np_array_length = np_array_to_be_added.shape[0]
                if cur_length < next_np_array_length:
                    np_array_to_be_added = np_array_to_be_added[next_np_array_length-cur_length:]
                else: 
                    addition_validation = addition_validation[cur_length-next_np_array_length:]
                addition_validation = np.add(addition_validation,np_array_to_be_added)

            # adding test
            if addition_test is None:
                addition_test = concatenated_results[ticker][feature][imf_level]['predicted_test'].values
            else:
                np_array_to_be_added = concatenated_results[ticker][feature][imf_level]['predicted_test'].values
                cur_length = addition_test.shape[0]
                next_np_array_length = np_array_to_be_added.shape[0]
                if cur_length < next_np_array_length:
                    np_array_to_be_added = np_array_to_be_added[next_np_array_length-cur_length:]
                else: 
                    addition_test = addition_test[cur_length-next_np_array_length:]
                addition_test = np.add(addition_test,np_array_to_be_added)
        
        scaler = scalers[ticker][feature]

        final_prediction_results[ticker][feature] = {
            'train_predicted': scaler.inverse_transform(addition_train.reshape(-1,1)),
            'validation_predicted': scaler.inverse_transform(addition_validation.reshape(-1,1)),
            'test_predicted': scaler.inverse_transform(addition_test.reshape(-1,1))
        }
        


ValueError: Expected 2D array, got 1D array instead:
array=[0.96396847 0.96657093 0.9837882  0.99616417 0.99161411 0.92707966
 0.84833842 0.78411862 0.73638716 0.71131171 0.66373848 0.62136977
 0.59136921 0.56311231 0.53433698 0.52052492 0.51329628 0.50060312
 0.48844638 0.47698653 0.46378375 0.45783361 0.45106798 0.4535817
 0.46302509 0.46512954 0.47023992 0.47408017 0.47300983 0.47787019
 0.49231237 0.513633   0.53064806 0.53768917 0.54894238 0.56832348
 0.58500769 0.58254974 0.58365204 0.58905303 0.59671016 0.6005969
 0.59405163 0.5792882  0.57876159 0.59331556 0.61648633 0.63921424
 0.65385845 0.65728572 0.65799403 0.65416119 0.6522391  0.65855646
 0.67728858 0.69678768 0.71034374 0.71144012 0.70618685 0.69696772
 0.68747493 0.68795654 0.68874968 0.6831349  0.66733431 0.6450518
 0.62443518 0.60193339 0.58928872 0.57994891 0.57943002 0.57967343
 0.57260969 0.57359538 0.58223973 0.59890369 0.60950603 0.60416319
 0.60810824 0.61417019 0.6174634  0.61948044 0.6201492  0.61123554
 0.601

In [11]:
final_prediction_results['PETR4'].keys()

dict_keys([])